In [25]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np


# Skeleton Specific Functions

In [26]:
def filter_distance_from_skeleton( skeleton_vertices,
                            child_mesh_verts,
                             child_mesh_faces,
                             distance_threshold=500,
                             significance_threshold=500,
                             n_sample_points=3, #currently this does not do anything
                             print_flag=False,
                             return_verts_faces=False):
    
    import time
    global_time = time.time()
    start_time = time.time()
    mesh_tree = KDTree(skeleton_vertices)
    distances,closest_node = mesh_tree.query(child_mesh_verts)
    if print_flag:
        print(f"Total time for KDTree creation and queries: {time.time() - start_time}")
    
    if print_flag:
        print("Original number vertices in child mesh = " + str(len(child_mesh_verts)))
    vertex_indices = np.where(distances > distance_threshold)[0]
    if print_flag:
        print("Number of vertices after distance threshold applied =  " + str(len(vertex_indices)))
    
    #gets the faces after has the vertices
    start_time = time.time()
    set_vertex_indices = set(list(vertex_indices))
    face_indices_lookup = np.linspace(0,len(child_mesh_faces)-1,len(child_mesh_faces)).astype('int')
    face_indices_lookup_bool = [len(set_vertex_indices.intersection(set(tri))) > 0 for tri in child_mesh_faces]
    face_indices_lookup = face_indices_lookup[face_indices_lookup_bool]

    if print_flag:
        print(f"Total time for finding faces after distance threshold applied: {time.time() - start_time}")
    if len(child_mesh_verts)<=0 or len(child_mesh_faces)<=0 or len(face_indices_lookup)<= 0:
        print("inside boolean function and returning because child faces are 0")
        return []
    
    start_time = time.time()
    trimesh_original = trimesh.Trimesh(child_mesh_verts,child_mesh_faces,process=False) 
    new_submesh = trimesh_original.submesh([face_indices_lookup],only_watertight=False,append=True)
    
    
    if print_flag:
        print("------Starting the mesh filter for significant outside pieces-------")

    mesh_pieces = new_submesh.split(only_watertight=False)
    
    if print_flag:
        print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    if print_flag:
        print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []
    
    #arrange the significant pieces from largest to smallest
    x = [len(k.vertices) for k in significant_pieces]
    sorted_indexes = sorted(range(len(x)), key=lambda k: x[k])
    sorted_indexes = sorted_indexes[::-1]
    sorted_significant_pieces = [significant_pieces[k] for k in sorted_indexes]
    
    return sorted_significant_pieces

In [57]:
"""
Extra skeleton functions:
"""
import networkx as nx
import time 
import numpy as np
import trimesh
import random

def generate_surface_skeleton(vertices, faces, surface_samples):
    
    #return surface_with_poisson_skeleton,path_length
    
    current_mesh = trimesh.Trimesh(vertices=vertices,
                                  faces = faces)


    start_time = time.time()

    # test on a sphere mesh
    mesh = current_mesh

    ga = nx.from_edgelist(mesh.edges)

    if surface_samples<len(vertices):
        k = surface_samples
    else:
        k = len(vertices)
    sampled_nodes = random.sample(ga.nodes, k)


    lp_end_list = []
    lp_magnitude_list = []

    for s in sampled_nodes: 
        sp_dict = nx.single_source_shortest_path_length(ga,s)

        list_keys = list(sp_dict.keys())
        longest_path_node = list_keys[len(list_keys)-1]
        longest_path_magnitude = sp_dict[longest_path_node]


        lp_end_list.append(longest_path_node)
        lp_magnitude_list.append(longest_path_magnitude)

    #construct skeleton from shortest path
    final_start = sampled_nodes[np.argmax(lp_magnitude_list)]
    final_end = sampled_nodes[np.argmax(lp_end_list)]

    node_list = nx.shortest_path(ga,final_start,final_end)
    if len(node_list) < 2:
        print("node_list len < 2 so returning empty list")
        return np.array([])
    #print("node_list = " + str(node_list))
    final_skeleton = mesh.vertices[np.vstack([node_list[:-1],node_list[1:]]).T]
    print(f"   Final Time for surface skeleton with sample size = {k} = {time.time() - start_time}")

    return final_skeleton

def save_skeleton_cgal(surface_with_poisson_skeleton,largest_mesh_path):
    """
    surface_with_poisson_skeleton (np.array) : nx2 matrix with the nodes
    """
    first_node = surface_with_poisson_skeleton[0][0]
    end_nodes =  surface_with_poisson_skeleton[:,1]
    
    skeleton_to_write = str(len(end_nodes) + 1) + " " + str(first_node[0]) + " " +  str(first_node[1]) + " " +  str(first_node[2])
    
    for node in end_nodes:
        skeleton_to_write +=  " " + str(node[0]) + " " +  str(node[1]) + " " +  str(node[2])
    
    output_file = largest_mesh_path
    if output_file[-5:] != ".cgal":
        output_file += ".cgal"
        
    f = open(output_file,"w")
    f.write(skeleton_to_write)
    f.close()
    return 

#read in the skeleton files into an array
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)
    unique_skeleton_verts = bone_array_total.reshape(-1,3)

def calculate_skeleton_distance(my_skeleton):
    total_distance = np.sum(np.sqrt(np.sum((my_skeleton[:,0] - my_skeleton[:,1])**2,axis=1)))
    return float(total_distance)



"""
Testing if the read and write are correct:

import numpy as np
file_name = "107816118160698192_leftover_1-0_skeleton.cgal"
my_skeleton = read_skeleton_revised(file_name)
my_skeleton
practice_file = "test_manual_cgal.cgal"
save_skeleton_cgal(my_skeleton,practice_file)
my_skeleton_2 = read_skeleton_revised(practice_file)

"""

"""
Pseudocode:
1) Get the number of rows to be even (by doing nothing or adding the first one to the final list
2) Take every other left neuorn, 1 + Take every other right neuron


A B
B C
C D
D E
E F
F G

Turns into

A C
C E
E G
"""

def downsample_skeleton(current_skeleton):
    #print("current_skeleton = " + str(current_skeleton.shape))
    """
    Downsamples the skeleton by 50% number of edges
    """
    extra_segment = []
    if current_skeleton.shape[0] % 2 != 0:
        extra_segment = np.array([current_skeleton[0]])
        current_skeleton = current_skeleton[1:]
        #print("extra_segment = " + str(extra_segment))
        #print("extra_segment.shape = " + str(extra_segment.shape))
    else:
        #print("extra_segment = " + str(extra_segment))
        pass

    even_indices = [k for k in range(0,current_skeleton.shape[0]) if k%2 == 0]
    odd_indices = [k for k in range(0,current_skeleton.shape[0]) if k%2 == 1]
    even_verts = current_skeleton[even_indices,0,:]
    odd_verts = current_skeleton[odd_indices,1,:]

    downsampled_skeleton = np.hstack([even_verts,odd_verts]).reshape(even_verts.shape[0],2,3)
    #print("dowsampled_skeleton.shape = " + str(downsampled_skeleton.shape))
    if len(extra_segment) > 0:
        #print("downsampled_skeleton = " + str(downsampled_skeleton.shape))
        final_downsampled_skeleton = np.vstack([extra_segment,downsampled_skeleton])
    else:
        final_downsampled_skeleton = downsampled_skeleton
    return final_downsampled_skeleton

In [45]:
# Calling the actual function
#main_mesh = trimesh.load_mesh("Surfae_Skeleton/107816118160698192_original_decimated.off")
main_mesh = trimesh.load_mesh("L5_neuron.off")
segment_id = "L5"

In [48]:
current_skeleton
downsampled_skeleton = downsample_skeleton(current_skeleton)

current_skeleton = (161, 2, 3)
extra_segment = [[[1123834.   991665.5  825632.4]
  [1123817.   991795.4  825841.4]]]
extra_segment.shape = (1, 2, 3)
downsampled_skeleton = (80, 2, 3)


# New function that can downsample the skeleton

In [29]:
import ipyvolume as ipv
current_skeleton = read_skeleton_revised("L5/L5-9-0_sWOp-0-0_skeleton.cgal")
current_skeleton
current_skeleton = current_skeleton[1:]
downsampled_skeleton = downsample_skeleton(current_skeleton)

# unique_skeleton_verts = np.vstack([current_skeleton[:,0,:],
#                                                np.array([current_skeleton[-1,1,:]])])
# skeleton_edges = np.vstack([np.array([range(0,len(current_skeleton)-1)]),
#                             np.array([range(1,len(current_skeleton))])]).T

unique_skeleton_verts = current_skeleton.reshape(-1,3)
skeleton_edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)

unique_skeleton_verts_downsampled = downsampled_skeleton.reshape(-1,3)
skeleton_edges_downsampled = np.arange(0,len(unique_skeleton_verts_downsampled)).astype("int").reshape(-1,2)

# print(unique_skeleton_verts)
# print(skeleton_edges)

ipv.figure()
mesh2 = ipv.plot_trisurf(unique_skeleton_verts_downsampled[:,0], 
                        unique_skeleton_verts_downsampled[:,1], 
                        unique_skeleton_verts_downsampled[:,2], 
                        lines=skeleton_edges_downsampled, color='blue')
mesh3 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=skeleton_edges, color='red')


original_mesh = mesh2
volume_maxs = np.max(unique_skeleton_verts,axis=0)
volume_mins = np.min(unique_skeleton_verts,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()


# mesh3 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
#                         unique_skeleton_verts[:,1], 
#                         unique_skeleton_verts[:,2], 
#                         lines=skeleton_edges, color='red')
# ipv.show()

# visualizing the unstitched mesh and skeleton

In [30]:
import os
name2 = "L5"
cgal_skeleton_file_list = []

for file in os.listdir(name2):
    if file.endswith(".cgal"):
        cgal_skeleton_file_list.append(str(os.path.join(name2, file)))
if len(cgal_skeleton_file_list) <= 0:
    #check to see if cgal list is empty
    raise Exception("There were no cgal files generated in process")

bone_array_total = np.vstack([read_skeleton_revised(k) 
                              for k in cgal_skeleton_file_list])

unique_skeleton_verts = bone_array_total.reshape(-1,3)
edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


In [31]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure(figsize=(15,15))
# we draw the tetrahedron

original_mesh_verts = main_mesh.vertices
original_mesh_faces = main_mesh.faces
edges = edges
unique_skeleton_verts = unique_skeleton_verts


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [32]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure(figsize=(15,15))
# we draw the tetrahedron

original_mesh_verts = main_mesh.vertices
original_mesh_faces = main_mesh.faces
edges = edges
unique_skeleton_verts = unique_skeleton_verts


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


# and also mark the vertices
mesh = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [33]:
# Find the difference in mesh in order to do for the second pass
returned_mesh = filter_distance_from_skeleton( unique_skeleton_verts,
                            main_mesh.vertices,
                             main_mesh.faces,
                             distance_threshold=1900,
                             significance_threshold=200,
                             n_sample_points=3, #currently this does not do anything
                             print_flag=False,
                             return_verts_faces=False)
total_leftover_mesh = trimesh.Trimesh(vertices=[],faces=[])
for rm in returned_mesh:
    total_leftover_mesh += rm

In [34]:
"""graph the still left over mesh"""
ipv.figure(figsize=(15,15))

mesh = ipv.plot_trisurf(total_leftover_mesh.vertices[:,0],
                       total_leftover_mesh.vertices[:,1],
                       total_leftover_mesh.vertices[:,2],
                       triangles=total_leftover_mesh.faces,color="orange")
mesh.color = [1.,0.,0.,1]

mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

mesh2.color = [0,0.,1,1]
mesh2.material.transparent = True

mesh3 = ipv.plot_trisurf(main_mesh.vertices[:,0],
                       main_mesh.vertices[:,1],
                       main_mesh.vertices[:,2],
                       triangles=main_mesh.faces)
mesh3.color = [0.,1.,0.,0.2]
mesh3.material.transparent = True


volume_max = np.max(total_leftover_mesh.vertices,axis=0)
volume_min = np.min(total_leftover_mesh.vertices,axis=0)

ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

# Going to rerun the skeletonization but just with the leftover mesh and the surface skeletonization

In [21]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(vertices=[], 
                                triangles=[],
                                neuron_ID="None",
                                folder="None",
                               path_and_filename="-1"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    if path_and_filename == "-1":
        #get the current file location
        if folder == "None":
            file_loc = pathlib.Path.cwd()
            
        else:
            file_loc = pathlib.Path.cwd() / folder
            
        filename = "neuron_" + str(neuron_ID)
        path_and_filename = file_loc / filename
    
    #print("path_and_filename = " + str(path_and_filename))
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    #print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename)#,str(filename),str(file_loc)

In [54]:
def recursive_skeletonizing(verts,faces,current_name,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth,
                            skeletonization_type,
                            surface_samples=200,
                            max_depth_cgal = 8,
                            max_depth_surface = 10,
                            min_skeleton_distance = 30,
                           boolean_distance_threshold=1000,
                            boolean_significance_threshold=1000,
                            boolean_n_sample_points=10,
                            boolean_print_flag = False,
                           largest_mesh_significant=3000,
                           list_of_meshes=[]):
    
    if len(list_of_meshes):
        mesh_pieces=list_of_meshes

        print(f" -->  Total significant pieces left: {[len(k.vertices) for k in mesh_pieces]}")
        # check if the number of 
        for j,piece in enumerate(mesh_pieces):
            new_current_name = current_name + "-" + str(j)
            print(f"Starting recursive call for size vertices of {len(piece.vertices)}")
            recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name,
                                   directory=directory,current_base_path=current_base_path,
                                    mesh_base_path=mesh_base_path,folder_name=folder_name,
                                   current_depth=current_depth,
                                    skeletonization_type=skeletonization_type,
                                    surface_samples=surface_samples,
                                    max_depth_cgal = max_depth_cgal,
                                    max_depth_surface = max_depth_surface,
                                    min_skeleton_distance = min_skeleton_distance,
                                   boolean_distance_threshold=boolean_distance_threshold,
                                    boolean_significance_threshold=boolean_significance_threshold,
                                    boolean_n_sample_points=boolean_n_sample_points,
                                    boolean_print_flag = boolean_print_flag,
                                   largest_mesh_significant=largest_mesh_significant)
        return
        
    else:
        print(f"\n----------Starting {current_name}----------")
        global_time = time.time()
        #print out the off file
        if current_depth >= max_depth_surface:
            print(f"\n\n*********Reamched maximum depth of {max_depth_surface} so returning********\n\n")
        if skeletonization_type== "cgal" and current_depth >= max_depth_cgal:
            print("     Reached max depth cgal")

            skeletonization_type = "surface_with_poisson"
            current_name += "_sWp"

            recursive_skeletonizing(verts,faces,current_name,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth=current_depth,
                            skeletonization_type=skeletonization_type,
                            surface_samples=surface_samples,
                            max_depth_cgal = max_depth_cgal,
                            max_depth_surface = max_depth_surface,
                            min_skeleton_distance = min_skeleton_distance,
                           boolean_distance_threshold=boolean_distance_threshold,
                            boolean_significance_threshold=boolean_significance_threshold,
                            boolean_n_sample_points=boolean_n_sample_points,
                            boolean_print_flag = boolean_print_flag,
                           largest_mesh_significant=largest_mesh_significant)




        if skeletonization_type == "surface_without_poisson":
            surface_without_poisson_skeleton = generate_surface_skeleton(verts,
                                                                          faces=faces,
                                                                          surface_samples = surface_samples)
            if len(surface_without_poisson_skeleton)>2:
                surface_without_poisson_skeleton = downsample_skeleton(surface_without_poisson_skeleton)
            if len(surface_without_poisson_skeleton) <= 0 :
                print("Returning because the skeleton returned was of size 0")
                return
            #print(f"Path length for surface skeleton WITHOUT poisson = {path_length}")

            largest_mesh_path = mesh_base_path + current_name

            #save off the skeleton
            save_skeleton_cgal(surface_without_poisson_skeleton,largest_mesh_path + "_skeleton")


        else:

            initial_output_path = mesh_base_path + current_name

            write_Whole_Neuron_Off_file(vertices=verts, 
                                        triangles=faces,
                                       path_and_filename=initial_output_path)


            # do the poisson surface reconstruction

            script_name = "poisson_working_meshlab.mls"

            input_file_base = initial_output_path
            output_file = input_file_base + "_poisson"
            meshlab_script_path_and_name =current_base_path + script_name

            start_time = time.time()
            print("     Starting Screened Poisson")
            meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                               output_path_and_filename=output_file + ".off",
                                                             meshlab_script=meshlab_script_path_and_name)
            print(f"     Total_time for Screended Poisson = {time.time() - start_time}")

            #2) Filter away for largest_poisson_piece:

            new_mesh = trimesh.load_mesh(output_file + ".off")
            mesh_splits = new_mesh.split(only_watertight=False)
            mesh_lengths = np.array([len(split.faces) for split in mesh_splits])
            largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
            largest_mesh = mesh_splits[largest_index][0]

            print("len(largest_mesh.vertices) = " + str(len(largest_mesh.vertices)))
            #4) If not of a significant size then return (Add the largest_poisson_piece to the Surface_Reconstruction_list and say that it was returned)
            if len(largest_mesh.vertices) < largest_mesh_significant:
                print(current_name + " largest mesh not significant AFTER POISSON RECONSTRUCTION")
                print("trying surface skeletonization without Poisson")

                """3) if no significant poisson piece --> do surface_with_poisson process
                    change the name slightly
                    change the skeletonization type"""

                skeletonization_type = "surface_without_poisson"
                current_name += "_sWOp"

                recursive_skeletonizing(verts,faces,current_name,
                                directory,current_base_path,mesh_base_path,folder_name,
                                current_depth=current_depth,
                                skeletonization_type=skeletonization_type,
                                surface_samples=surface_samples,
                                max_depth_cgal = max_depth_cgal,
                                max_depth_surface = max_depth_surface,
                                min_skeleton_distance = min_skeleton_distance,
                               boolean_distance_threshold=boolean_distance_threshold,
                                boolean_significance_threshold=boolean_significance_threshold,
                                boolean_n_sample_points=boolean_n_sample_points,
                                boolean_print_flag = boolean_print_flag,
                               largest_mesh_significant=largest_mesh_significant)
                return 

            if skeletonization_type == "surface_with_poisson":
                surface_with_poisson_skeleton = generate_surface_skeleton(largest_mesh.vertices,
                                                                          faces=largest_mesh.faces,
                                                                          surface_samples = surface_samples)

                if len(surface_with_poisson_skeleton) <= 0 :
                    print("Returning because the skeleton returned was of size 0")
                    return
                #print(f"Path length for surface skeleton with poisson = {path_length}")

                largest_mesh_path = mesh_base_path + current_name

                #save off the skeleton
                save_skeleton_cgal(surface_with_poisson_skeleton,largest_mesh_path + "_skeleton")


            elif skeletonization_type == "cgal":

                #5) If significant size output the mesh
                largest_mesh_path = mesh_base_path + current_name
                write_Whole_Neuron_Off_file(vertices=largest_mesh.vertices, 
                                            triangles=largest_mesh.faces,
                                           path_and_filename=largest_mesh_path)


                #6) Run skeletonization on it:
                start_time = time.time()
                print("     Starting Calcification")
                cm.calcification(largest_mesh_path)
                print(f"     Total_time for Calcification = {time.time() - start_time}")




        """ ***** GOING TO USE THE SKELETON VERTICES INSTEAD OF THE MESH VERTICES"""


        #7) Subtract skeleton vertices from current mesh
        start_time = time.time()
        print("     Starting Mesh boolean difference")

        cgal_skeleton_file = largest_mesh_path + "_skeleton.cgal"

        try:
            returned_skeleton = read_skeleton_revised(cgal_skeleton_file)
        except OSError as e:
            print("\n\n**** No cgal file was found so returning ******\n\n")
            return
        else:
            pass
            #print("\n\n**** unknown error ocured when reading in cgal file so returning ******\n\n")
            #return


        skeletal_length = calculate_skeleton_distance(returned_skeleton)
        print(f" Skeletal Length = {skeletal_length}  ")
        # returning based on skeletal length: 
        if skeletal_length <= min_skeleton_distance:

            if skeletonization_type == "cgal":
                print("Skeleton generated was too small --> cgal skeletonization turning into surface with poisson")
                #try the surface skeletonization with poisson
                current_name += "sWp"
                skeletonization_type = "surface_with_poisson"

            elif skeletonization_type == "surface_with_poisson":
                print("Skeleton generated was too small --> surface WITH poisson turning into surface WTIHOUT poisson")
                current_name += "sWOp"
                skeletonization_type = "surface_without_poisson"
            else:
                return

            recursive_skeletonizing(verts,faces,current_name,
                        directory,current_base_path,mesh_base_path,folder_name,
                        current_depth=current_depth,
                        skeletonization_type=skeletonization_type,
                        surface_samples=surface_samples,
                        max_depth_cgal = max_depth_cgal,
                        max_depth_surface = max_depth_surface,
                        min_skeleton_distance = min_skeleton_distance,
                       boolean_distance_threshold=boolean_distance_threshold,
                        boolean_significance_threshold=boolean_significance_threshold,
                        boolean_n_sample_points=boolean_n_sample_points,
                        boolean_print_flag = boolean_print_flag,
                       largest_mesh_significant=largest_mesh_significant)
            return




        skeleton_vertices = np.vstack([returned_skeleton[:,0],returned_skeleton[:,1]])
        #getting the boolean difference
    #     mesh_pieces =neuron_boolean_difference( largest_mesh.vertices,
    #                                    largest_mesh.faces,
    #                                     verts,
    #                                    faces,
    #                                    distance_threshold=boolean_distance_threshold,
    #                                    significance_threshold=boolean_significance_threshold,
    #                                    n_sample_points=boolean_n_sample_points,
    #                                   print_flag = boolean_print_flag)

    #     mesh_pieces =neuron_boolean_difference( skeleton_vertices
    #                                 verts,
    #                                faces,
    #                                distance_threshold=boolean_distance_threshold,
    #                                significance_threshold=boolean_significance_threshold,
    #                                n_sample_points=boolean_n_sample_points,
    #                               print_flag = boolean_print_flag)

        if skeletonization_type == "cgal":
            current_boolean_distance_threshold = boolean_distance_threshold
        else:
            current_boolean_distance_threshold = 500

        mesh_pieces = filter_distance_from_skeleton( skeleton_vertices,
                                verts,
                                 faces,
                                 distance_threshold=current_boolean_distance_threshold,
                                 significance_threshold=boolean_significance_threshold,
                                 n_sample_points=boolean_n_sample_points,
                                 print_flag=boolean_print_flag,
                                 return_verts_faces=False)



        print(f"     Total_time for Mesh boolean difference = {time.time() - start_time}")

        print(f"Total time for one mesh piece skeleton = {time.time() - global_time}")

        print(f"{current_name} there were {len(mesh_pieces)} significant pieces leftover after largest mesh")

        if len(mesh_pieces) <= 0:
            print(f"{current_name} returning because 0 significant pieces")
            return 



        #increment the current depth 
        current_depth = current_depth + 1

        print(f" -->  Total significant pieces left: {[len(k.vertices) for k in mesh_pieces]}")
        # check if the number of 
        for j,piece in enumerate(mesh_pieces):
            new_current_name = current_name + "-" + str(j)
            print(f"Starting recursive call for size vertices of {len(piece.vertices)}")
            recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name,
                                   directory=directory,current_base_path=current_base_path,
                                    mesh_base_path=mesh_base_path,folder_name=folder_name,
                                   current_depth=current_depth,
                                    skeletonization_type=skeletonization_type,
                                    surface_samples=surface_samples,
                                    max_depth_cgal = max_depth_cgal,
                                    max_depth_surface = max_depth_surface,
                                    min_skeleton_distance = min_skeleton_distance,
                                   boolean_distance_threshold=boolean_distance_threshold,
                                    boolean_significance_threshold=boolean_significance_threshold,
                                    boolean_n_sample_points=boolean_n_sample_points,
                                    boolean_print_flag = boolean_print_flag,
                                   largest_mesh_significant=largest_mesh_significant)

        return

In [38]:
import calcification_Module as cm
import time
import os
import pathlib

def calculate_skeleton_edges(input_verts,input_faces,segment_id,
                            boolean_distance_threshold=1000,
                                   boolean_significance_threshold=300,
                                   surface_samples=200,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=300,
                            skeletonization_type="surface_without_poisson",
                            list_of_meshes=[]):
    original_mesh = trimesh.Trimesh(vertices=input_verts,faces=input_faces)
    name2 = segment_id
    
    
    directory = "./" + name2
    if not os.path.exists(directory):
        os.makedirs(directory)


    current_base_path = str(pathlib.Path.cwd()) + "/"
    mesh_base_path = current_base_path + str(name2) + "/"
    folder_name = str(name2)

    current_layer = 1
    
#     recursive_skeletonizing(original_mesh.vertices,original_mesh.faces,name2,
#                             directory,current_base_path,mesh_base_path,folder_name,
#                         boolean_distance_threshold=boolean_distance_threshold,
#                             boolean_significance_threshold=boolean_significance_threshold,
#                             boolean_n_sample_points=boolean_n_sample_points,
#                             boolean_print_flag = boolean_print_flag,
#                            largest_mesh_significant=largest_mesh_significant)
    
    
    recursive_skeletonizing(original_mesh.vertices,original_mesh.faces,name2,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth=1,
                            skeletonization_type=skeletonization_type,
                            surface_samples=surface_samples,
                            max_depth_cgal = 8,
                            max_depth_surface = 10,
                            min_skeleton_distance = 10,
                           boolean_distance_threshold=boolean_distance_threshold,
                            boolean_significance_threshold=boolean_significance_threshold,
                            boolean_n_sample_points=surface_samples,
                            boolean_print_flag = False,
                            list_of_meshes=list_of_meshes
                           )
    
    
    
    
    cgal_skeleton_file_list = []
    
    for file in os.listdir(name2):
        if file.endswith(".cgal"):
            cgal_skeleton_file_list.append(str(os.path.join(name2, file)))
    if len(cgal_skeleton_file_list) <= 0:
        #check to see if cgal list is empty
        raise Exception("There were no cgal files generated in process")
    
    bone_array_total = np.vstack([read_skeleton_revised(k) 
                                  for k in cgal_skeleton_file_list])

    unique_skeleton_verts = bone_array_total.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


    total_edges = np.array([])

    for k in cgal_skeleton_file_list:
        bone_array = read_skeleton_revised(k) 

        #add the skeleton edges to the total edges
        if not total_edges.any():
            total_edges = bone_array
        else:
            total_edges = np.vstack([total_edges,bone_array])

    total_edges_stitched = stitch_skeleton_with_degree_check_vp2(total_edges)
    
    unique_skeleton_verts = total_edges_stitched.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)
    
#     #erase the folder with all of the data in it
#     import shutil
#     shutil.rmtree(directory)
    
    #return the edges
    return edges,unique_skeleton_verts

In [58]:
import time
total_start_time = time.time()

returned_mesh

segment_id = "Layer_5_Run_2"

edges,unique_skeleton_verts = calculate_skeleton_edges(main_mesh.vertices,main_mesh.faces,segment_id,
                                   boolean_distance_threshold=500,
                                   boolean_significance_threshold=250,
                                   surface_samples=500,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=250,
                                                      skeletonization_type="surface_without_poisson",
                                                      list_of_meshes=returned_mesh)
print(f"Total time for calculating skeleton = {time.time() - start_time}")

 -->  Total significant pieces left: [940, 640, 621, 523, 500, 493, 479, 478, 474, 473, 469, 467, 465, 462, 461, 456, 454, 451, 450, 440, 435, 433, 429, 428, 424, 417, 416, 414, 407, 402, 402, 395, 393, 392, 390, 389, 388, 381, 381, 379, 376, 375, 369, 369, 368, 353, 348, 347, 339, 339, 338, 336, 327, 325, 321, 309, 306, 305, 304, 304, 303, 302, 301, 298, 295, 294, 283, 281, 281, 280, 273, 273, 269, 267, 264, 264, 263, 262, 262, 258, 258, 254, 254, 252, 251, 249, 248, 240, 240, 233, 228, 226, 226, 224, 223, 218, 218, 216, 216, 213, 212, 210, 208, 207, 200, 199, 199, 197, 196, 196, 196, 190, 190, 188, 186, 184, 184, 182, 182, 180, 180, 179, 179, 177, 176, 176, 176, 175, 175, 175, 173, 170, 168, 168, 166, 165, 164, 163, 162, 161, 159, 158, 154, 153, 153, 152, 152, 151, 151, 143, 142, 138, 137, 136, 135, 132, 128, 127, 127, 125, 125, 125, 124, 120, 111, 111, 109]
Starting recursive call for size vertices of 940

----------Starting Layer_5_Run_2-0----------
   Final Time for surface skelet

NameError: name 'stitch_skeleton_with_degree_check_vp2' is not defined

# graph the still left over mesh

In [60]:
import os
name2 = ["L5","Layer_5_Run_2"]
cgal_skeleton_file_list = []

for n in name2:
    for file in os.listdir(n):
        if file.endswith(".cgal"):
            cgal_skeleton_file_list.append(str(os.path.join(n, file)))
    if len(cgal_skeleton_file_list) <= 0:
        #check to see if cgal list is empty
        raise Exception("There were no cgal files generated in process")

bone_array_total = np.vstack([read_skeleton_revised(k) 
                              for k in cgal_skeleton_file_list])

unique_skeleton_verts = bone_array_total.reshape(-1,3)
edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


In [61]:
# Graph the final result: 

ipv.figure(figsize=(15,15))

# mesh = ipv.plot_trisurf(total_leftover_mesh.vertices[:,0],
#                        total_leftover_mesh.vertices[:,1],
#                        total_leftover_mesh.vertices[:,2],
#                        triangles=total_leftover_mesh.faces,color="orange")
# mesh.color = [1.,0.,0.,1]

mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

mesh2.color = [0,0.,1,1]
mesh2.material.transparent = True

mesh3 = ipv.plot_trisurf(main_mesh.vertices[:,0],
                       main_mesh.vertices[:,1],
                       main_mesh.vertices[:,2],
                       triangles=main_mesh.faces)
mesh3.color = [0.,1.,0.,0.2]
mesh3.material.transparent = True


volume_max = np.max(total_leftover_mesh.vertices,axis=0)
volume_min = np.min(total_leftover_mesh.vertices,axis=0)

ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Doing the skeleton stitching: 

In [62]:
import time
import numpy as np
import datajoint as dj
import networkx as nx

import matplotlib.pyplot as plt #for graph visualization
from scipy.spatial import distance_matrix
from tqdm import tqdm

# reimplementation of connected_component_subgraphs deprecated in networkx>=2.4
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

def stitch_skeleton_with_degree_check_vp3(staring_edges,max_stitch_distance=18000,end_node=False):
    

    #unpacks so just list of vertices
    vertices_unpacked  = staring_edges.reshape(-1,3)

    #reduce the number of repeat vertices and convert to list
    unique_rows = np.unique(vertices_unpacked, axis=0)
    unique_rows_list = unique_rows.tolist()

    #assigns the number to the vertex (in the original vertex list) that corresponds to the index in the unique list
    vertices_unpacked_coefficients = [unique_rows_list.index(a) for a in vertices_unpacked.tolist()]

    #reshapes the vertex list to become an edge list
    edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
    
    #create the graph from the edges
    B = nx.Graph()
    B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
    B.add_edges_from(edges_with_coefficients)

    # find the shortest distance between the two different subgraphs:
    from scipy.spatial import distance_matrix

    UG = B.to_undirected()

    # extract subgraphs
    sub_graphs = connected_component_subgraphs(UG)

    subgraphs_list = []
    for i, sg in enumerate(sub_graphs):
        #print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
        #print("\tNodes:", sg.nodes(data=True))
        #print("\tEdges:", sg.edges())
        subgraphs_list.append(sg)

    print("len_subgraphs AT BEGINNING = " + str(len(subgraphs_list)))

    while len(subgraphs_list) > 1:
#         if subgraph_counter>=len(subgraphs_list):
#             print("Breaking because the sugraph counter was larger than list")
#             break
#         print("subgraph_counter = " + str(subgraph_counter))
        current_sub_graph = subgraphs_list[0]
        #get all of the coordinates of this subgraph 
        #get all of the coordinates
        
        #get all of the coordinates
        current_nodes = np.array(list(current_sub_graph.nodes))
        current_coord = nx.get_node_attributes(current_sub_graph,'coordinates')
        current_coordinates = np.array(list(current_coord.values()))
        
        
        
        
#         if end_node == True:
#             current_nodes = np.array([x for x,n in current_sub_graph.degree() if n <= 1])
#             current_coord = nx.get_node_attributes(current_sub_graph,'coordinates')
#             current_coord_filter = { k:v for k,v in current_coord.items() if k in  current_nodes}
#             current_coordinates = np.array(list(current_coord_filter.values()))
        
#         else:
#             #get all of the coordinates
#             current_nodes = np.array(list(current_sub_graph.nodes))
#             current_coord = nx.get_node_attributes(current_sub_graph,'coordinates')
#             current_coordinates = np.array(list(current_coord.values()))
            
    

        #store the minimum distance
        min_dist = np.inf
        min_dist_subgraph_index = -1
        min_dist_edge = [-1,-1]
        min_dist_edge_index = [-1,-1]

        #i = 0
        for i, sg in enumerate(subgraphs_list):
            if i == 0:
                continue

            #get all of the coordinates
            if end_node == True:
                new_nodes = np.array([x for x,n in sg.degree() if n <= 1])
                #print("new nodes = " + str(new_nodes))
                new_coord = nx.get_node_attributes(sg,'coordinates')
                #print("new_coord = " + str(new_coord))
                new_coord_filter = { k:v for k,v in new_coord.items() if k in  new_nodes}
                new_coordinates = np.array(list(new_coord_filter.values()))
            else:
                #get all of the coordinates
                new_nodes = np.array(list(sg.nodes))
                new_coord = nx.get_node_attributes(sg,'coordinates')
                new_coordinates = np.array(list(new_coord.values()))
                
            #find the closest distance between those points
            a_New = current_coordinates
            b_New = new_coordinates

            # print("starting distance matrix")
            start_time = time.time()

            a_b_distance = distance_matrix(a_New, b_New)
            #print(a_b_distance)

            current_min = np.min(a_b_distance)
            dist_matrix_index = np.where(a_b_distance == current_min)
            min_indexes = dist_matrix_index[0][0],dist_matrix_index[1][0]

            #print("Min indexes = " + str(min_indexes))

            if current_min<min_dist:
                min_dist = current_min
                min_dist_subgraph_index = i
                min_dist_edge_index = [current_nodes[min_indexes[0]],
                                       new_nodes[min_indexes[1]]]
                min_dist_edge = [current_coordinates[min_indexes[0]],
                                        new_coordinates[min_indexes[1]]]
            #i += 1

        #add the shortest connection to the overall edges
        print("min_dist = " + str(min_dist))
        print("min_dist_subgraph_index = " + str(min_dist_subgraph_index))
        print("min_dist_edge_index = " + str(min_dist_edge_index))
        print("min_dist_edge = " + str(min_dist_edge))
        
        #check if the distance is too far:
        if min_dist > max_stitch_distance:
            print("********** breaking because min distance is too large*****")
            break
           
        

        #adds the new edge to the graph
        B.add_edge(*min_dist_edge_index)

        #
        #print("edges after adding")
        #print(B.edges())
        UG = B.to_undirected()

        # extract subgraphs
        sub_graphs = connected_component_subgraphs(UG)
        subgraphs_list = []
        for i, sg in enumerate(sub_graphs):
#             print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
#             print("\tNodes:", sg.nodes(data=True))
#             print("\tEdges:", sg.edges())
            subgraphs_list.append(sg)

        print("len_subgraphs AT END= " + str(len(subgraphs_list)))
        
        
    # get the largest subgraph!!! in case have missing pieces
    
    #add all the new edges to the 
    Gc = max(connected_component_subgraphs(B), key=len)
    
    total_coord = nx.get_node_attributes(B,'coordinates')
    #print("len(total_coord)= " + str(len(total_coord)))
    current_coordinates = np.array(list(total_coord.values()))
    #print("current_coordinates.shape= " + str(current_coordinates.shape))
    #print("Gc.edges() = " + str(Gc.edges()))
    return current_coordinates[Gc.edges()]


"""
For finding the distances of the skeleton
"""
def find_skeleton_distance(example_edges):
    total_distance = np.sum([np.linalg.norm(a-b) for a,b in example_edges])
    return total_distance

from scipy.spatial import distance

def find_skeleton_distance_scipy(example_edges):
    total_distance = np.sum([distance.euclidean(a, b) for a,b in example_edges])
    return total_distance

In [63]:
total_edges_stitched = stitch_skeleton_with_degree_check_vp3(bone_array_total)

unique_skeleton_verts_final = total_edges_stitched.reshape(-1,3)
edges_final = np.arange(0,len(unique_skeleton_verts_final)).astype("int").reshape(-1,2)

len_subgraphs AT BEGINNING = 442
min_dist = 1856.5296792672202
min_dist_subgraph_index = 1
min_dist_edge_index = [26, 30]
min_dist_edge = [array([ 874534.6,  981713.1, 1066652. ]), array([ 876307. ,  981339.4, 1066245. ])]
len_subgraphs AT END= 441
min_dist = 2431.006992996935
min_dist_subgraph_index = 1
min_dist_edge_index = [43, 46]
min_dist_edge = [array([ 878121. ,  981766.2, 1065898. ]), array([ 880446. ,  981273.2, 1066409. ])]
len_subgraphs AT END= 440
min_dist = 1788.8262296825296
min_dist_subgraph_index = 2
min_dist_edge_index = [93, 110]
min_dist_edge = [array([ 888408.5,  983934.6, 1063641. ]), array([ 889868.3,  984497.8, 1062774. ])]
len_subgraphs AT END= 439
min_dist = 1745.7627444758782
min_dist_subgraph_index = 5
min_dist_edge_index = [238, 240]
min_dist_edge = [array([ 903181.2,  988539.8, 1058385. ]), array([ 904303.2,  989596.4, 1057565. ])]
len_subgraphs AT END= 438
min_dist = 1830.6341005236195
min_dist_subgraph_index = 4
min_dist_edge_index = [252, 236]
min_dist_e

KeyboardInterrupt: 

In [90]:
"""
Pseudocode for skeleton stitcher: 
-------Old method-------

1) creates a network with unique nodes and their coordinates and the edges of the graph
2) extract subgraphs: Appends the list of subgraphs
3) Iterates through all of the subgraphs
a. gets first subgraph
b. gets current coordinates of subgraph
c. strip these coordinates from all the list of entire coordinates
d. Find closest node
e. Add edge (make sure not greater than max length)
f. recompute the subgraphs
g. keep looping until only one component
4) Output the edges as a skeleton of coordinates



"""

from pykdtree.kdtree import KDTree

import scipy
def stitch_skeleton_with_degree_check_vp4(staring_edges,max_stitch_distance=18000,end_node=False):
    

    #unpacks so just list of vertices
    vertices_unpacked  = staring_edges.reshape(-1,3)

    #reduce the number of repeat vertices and convert to list
    unique_rows = np.unique(vertices_unpacked, axis=0)
    unique_rows_list = unique_rows.tolist()

    #assigns the number to the vertex (in the original vertex list) that corresponds to the index in the unique list
    vertices_unpacked_coefficients = np.array([unique_rows_list.index(a) for a in vertices_unpacked.tolist()])

    #reshapes the vertex list to become an edge list
    edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
    
    #create the graph from the edges
    B = nx.Graph()
    B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
    B.add_edges_from(edges_with_coefficients)

    # find the shortest distance between the two different subgraphs:
    from scipy.spatial import distance_matrix

    UG = B.to_undirected()

#     # extract subgraphs
#     sub_graphs = connected_component_subgraphs(UG)

#     subgraphs_list = []
#     for i, sg in enumerate(sub_graphs):
#         #print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
#         #print("\tNodes:", sg.nodes(data=True))
#         #print("\tEdges:", sg.edges())
#         subgraphs_list.append(sg)

    #get all of the coordinates

    print("len_subgraphs AT BEGINNING of the loop")
    counter = 0
    print_flag = True
    while True:
        counter+= 1
        if print_flag:
            print(f"Starting Loop {counter}")
        start_time = time.time()
        """
        1) Get the indexes of the subgraph
        2) Build a KDTree from those not in the subgraph (save the vertices of these)
        3) Query against the nodes in the subgraph  and get the smallest distance
        4) Create this new edge
        
        """
        
        #1) Get the indexes of the subgraph
        n_components, labels = scipy.sparse.csgraph.connected_components(csgraph=nx.adjacency_matrix(UG), directed=False, return_labels=True)
        subgraph_components = np.where(labels==0)[0]
        #print("subgraph_components = " + str(subgraph_components))
        if len(subgraph_components) == len(UG.nodes):
            print("all graph is one component!")
            break

        outside_components = np.where(labels !=0)[0]
        #print("outside_components = " + str(outside_components))
        
        #2) Build a KDTree from those not in the subgraph (save the vertices of these)
        mesh_tree = KDTree(vertices_unpacked_coefficients[outside_components])

        
        #3) Query against the nodes in the subgraph  and get the smallest distance
        """
        Conclusion:
        Distance is of the size of the parts that are in the KDTree
        The closest nodes represent those that were queryed

        """
        distances,closest_node = mesh_tree.query(vertices_unpacked_coefficients[subgraph_components])
        min_index = np.argmin(distances)
        closest_subgraph_node = subgraph_components[closest_node[min_index]]
        closest_outside_node = outside_components[min_index]
        
        #4) Create this new edge
        UG.add_edge(closest_subgraph_node,closest_outside_node)
        
        if print_flag:
            print(f"Total Time for loop = {time.time() - start_time}")
    
        
    # get the largest subgraph!!! in case have missing pieces
    
    #add all the new edges to the 
    Gc = max(connected_component_subgraphs(B), key=len)
    
    total_coord = nx.get_node_attributes(UG,'coordinates')

    current_coordinates = np.array(list(total_coord.values()))

    return current_coordinates[Gc.edges()]


"""
For finding the distances of the skeleton
"""
def find_skeleton_distance(example_edges):
    total_distance = np.sum([np.linalg.norm(a-b) for a,b in example_edges])
    return total_distance

from scipy.spatial import distance

def find_skeleton_distance_scipy(example_edges):
    total_distance = np.sum([distance.euclidean(a, b) for a,b in example_edges])
    return total_distance

In [91]:
total_edges_stitched = stitch_skeleton_with_degree_check_vp4(bone_array_total)

unique_skeleton_verts_final = total_edges_stitched.reshape(-1,3)
edges_final = np.arange(0,len(unique_skeleton_verts_final)).astype("int").reshape(-1,2)

len_subgraphs AT BEGINNING of the loop
Starting Loop 1
Total Time for loop = 0.33653712272644043
Starting Loop 2
Total Time for loop = 0.17330050468444824
Starting Loop 3
Total Time for loop = 0.18644285202026367
Starting Loop 4
Total Time for loop = 0.3270072937011719
Starting Loop 5
Total Time for loop = 0.33197784423828125
Starting Loop 6
Total Time for loop = 0.597360372543335
Starting Loop 7
Total Time for loop = 0.3092041015625
Starting Loop 8
Total Time for loop = 0.2407979965209961
Starting Loop 9
Total Time for loop = 0.46753835678100586
Starting Loop 10
Total Time for loop = 0.2110743522644043
Starting Loop 11
Total Time for loop = 0.19522380828857422
Starting Loop 12
Total Time for loop = 0.33295369148254395
Starting Loop 13
Total Time for loop = 0.18618488311767578
Starting Loop 14
Total Time for loop = 0.19455218315124512
Starting Loop 15
Total Time for loop = 0.34424448013305664
Starting Loop 16
Total Time for loop = 0.19668292999267578
Starting Loop 17
Total Time for loo

IndexError: index 29 is out of bounds for axis 0 with size 27

In [111]:
random_queries = np.random.uniform(0,1,(9,3))
random_queries

array([[0.32547681, 0.45730816, 0.83576217],
       [0.10342191, 0.2564919 , 0.7845737 ],
       [0.47241876, 0.5765472 , 0.39160584],
       [0.96290634, 0.94675918, 0.91078484],
       [0.63660696, 0.80867956, 0.35518191],
       [0.42639235, 0.68353082, 0.39451197],
       [0.83452409, 0.96480333, 0.16494052],
       [0.58168145, 0.99172452, 0.94634509],
       [0.65714056, 0.49133701, 0.00335787]])

In [113]:
mesh_tree = KDTree(total_coordinates[outside_components])
distances,closest_node = mesh_tree.query(random_queries)
closest_node

array([2315, 2315, 2315, 2315, 2315, 2315, 2315, 2315, 2315], dtype=uint32)

In [115]:
staring_edges = bone_array_total
max_stitch_distance=18000
end_node=False

#unpacks so just list of vertices
vertices_unpacked  = staring_edges.reshape(-1,3)

#reduce the number of repeat vertices and convert to list
unique_rows = np.unique(vertices_unpacked, axis=0)
unique_rows_list = unique_rows.tolist()

total_coordinates = unique_rows

#assigns the number to the vertex (in the original vertex list) that corresponds to the index in the unique list
vertices_unpacked_coefficients = np.array([unique_rows_list.index(a) for a in vertices_unpacked.tolist()])

#reshapes the vertex list to become an edge list
edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)

#create the graph from the edges
B = nx.Graph()
B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
B.add_edges_from(edges_with_coefficients)

# find the shortest distance between the two different subgraphs:
from scipy.spatial import distance_matrix

UG = B.to_undirected()

#     # extract subgraphs
#     sub_graphs = connected_component_subgraphs(UG)

#     subgraphs_list = []
#     for i, sg in enumerate(sub_graphs):
#         #print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
#         #print("\tNodes:", sg.nodes(data=True))
#         #print("\tEdges:", sg.edges())
#         subgraphs_list.append(sg)

#get all of the coordinates

print("len_subgraphs AT BEGINNING of the loop")
counter = 0
print_flag = True
while True:
    counter+= 1
    if print_flag:
        print(f"Starting Loop {counter}")
    start_time = time.time()
    """
    1) Get the indexes of the subgraph
    2) Build a KDTree from those not in the subgraph (save the vertices of these)
    3) Query against the nodes in the subgraph  and get the smallest distance
    4) Create this new edge

    """

    #1) Get the indexes of the subgraph
    n_components, labels = scipy.sparse.csgraph.connected_components(csgraph=nx.adjacency_matrix(UG), directed=False, return_labels=True)
    subgraph_components = np.where(labels==0)[0]
    #print("subgraph_components = " + str(subgraph_components))
    if len(subgraph_components) == len(UG.nodes):
        print("all graph is one component!")
        break

    outside_components = np.where(labels !=0)[0]
    #print("outside_components = " + str(outside_components))

    #2) Build a KDTree from those not in the subgraph (save the vertices of these)
    mesh_tree = KDTree(total_coordinates[outside_components])


    #3) Query against the nodes in the subgraph  and get the smallest distance
    """
    Conclusion:
    Distance is of the size of those queryed
    The closest nodes represent those that were originally used to set the KD tree

    """
    #print("vertices_unpacked_coefficients[subgraph_components] = " + str(total_coordinates[subgraph_components]))
    distances,closest_node = mesh_tree.query(total_coordinates[subgraph_components])
    min_index = np.argmin(distances)
    closest_subgraph_node = subgraph_components[min_index]
    closest_outside_node = outside_components[closest_node[min_index]]

    #4) Create this new edge
    UG.add_edge(closest_subgraph_node,closest_outside_node)

    if print_flag:
        print(f"Total Time for loop = {time.time() - start_time}")


# get the largest subgraph!!! in case have missing pieces

#add all the new edges to the 
Gc = max(connected_component_subgraphs(B), key=len)

total_coord = nx.get_node_attributes(UG,'coordinates')

current_coordinates = np.array(list(total_coord.values()))

final_result = current_coordinates[Gc.edges()]

len_subgraphs AT BEGINNING of the loop
Starting Loop 1
Total Time for loop = 0.21888279914855957
Starting Loop 2
Total Time for loop = 0.3619096279144287
Starting Loop 3
Total Time for loop = 0.19794416427612305
Starting Loop 4
Total Time for loop = 0.34850335121154785
Starting Loop 5
Total Time for loop = 0.1994493007659912
Starting Loop 6
Total Time for loop = 0.2084333896636963
Starting Loop 7
Total Time for loop = 0.34737682342529297
Starting Loop 8
Total Time for loop = 0.19383478164672852
Starting Loop 9
Total Time for loop = 0.24681711196899414
Starting Loop 10
Total Time for loop = 0.35263609886169434
Starting Loop 11
Total Time for loop = 0.20081019401550293
Starting Loop 12
Total Time for loop = 0.3365664482116699
Starting Loop 13
Total Time for loop = 0.18099498748779297
Starting Loop 14
Total Time for loop = 0.1998605728149414
Starting Loop 15
Total Time for loop = 0.3371450901031494
Starting Loop 16
Total Time for loop = 0.20265698432922363
Starting Loop 17
Total Time for 

In [121]:
total_coordinates[UG.edges].shape

(30941, 2, 3)

In [123]:
total_edges_stitched

(30941, 2, 3)

In [124]:
total_edges_stitched = total_coordinates[UG.edges]

unique_skeleton_verts_final = total_edges_stitched.reshape(-1,3)
edges_final = np.arange(0,len(unique_skeleton_verts_final)).astype("int").reshape(-1,2)

In [118]:
total_edges_stitched.shape

(897, 2, 3)

# Graph and see how good the skeleton is:

In [128]:
x = 4

In [129]:
# Graph the final result: 

ipv.figure(figsize=(15,15))

mesh2 = ipv.plot_trisurf(unique_skeleton_verts_final[:,0], 
                        unique_skeleton_verts_final[:,1], 
                        unique_skeleton_verts_final[:,2], 
                        lines=edges_final, color='blue')

mesh2.color = [0,0.,1,1]
mesh2.material.transparent = True

mesh3 = ipv.plot_trisurf(main_mesh.vertices[:,0],
                       main_mesh.vertices[:,1],
                       main_mesh.vertices[:,2],
                       triangles=main_mesh.faces)
mesh3.color = [0.,1.,0.,0.2]
mesh3.material.transparent = True


volume_max = np.max(total_leftover_mesh.vertices,axis=0)
volume_min = np.min(total_leftover_mesh.vertices,axis=0)

ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])

ipv.style.set_style_light()
ipv.style.box_off()
ipv.style.axes_off()

ipv.show()


In [127]:
np.savez("Final_L5_skeleton_and_mesh.npz",unique_skeleton_verts_final=unique_skeleton_verts_final,edges_final=edges_final)
       